### TIme to Query Data From Athena

In [2]:
%store -r ingest_create_athena_table_csv_passed

In [3]:
try:
    ingest_create_athena_table_csv_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")

In [4]:
print(ingest_create_athena_table_csv_passed)

True


In [5]:
if not ingest_create_athena_table_csv_passed:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not register the TSV Data.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")
else:
    print("[OK]")

[OK]


In [6]:
%store

Stored variables and their in-db values:
data_path                                         -> '/root/AAI-540/Module2/csv'
ingest_create_athena_db_mod2_passed               -> True
ingest_create_athena_table_csv_passed             -> True
s3_private_path_csv                               -> 's3://sagemaker-us-east-1-004608622582/module2_dat
setup_dependencies_mod2_passed                    -> True
setup_s3_bucket_passed                            -> True


### Setup

In [7]:
import sagemaker
import boto3

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [8]:
import awswrangler as wr

### Query From Glue Catalog

In [9]:
database_name = "mod2_db"
table_name_csv = "music"

In [10]:
for table in wr.catalog.get_tables(database="mod2_db"):
    print(table["Name"])

music


### Query From Athena

In [11]:
%%time
df = wr.athena.read_sql_query(sql="SELECT * FROM {}.{} LIMIT 5000".format(database_name,table_name_csv), database=database_name)

CPU times: user 456 ms, sys: 48.9 ms, total: 505 ms
Wall time: 2.82 s


In [12]:
df.head()

,track_id,artists,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,73,230666,False,0.676,0.4610,1,-6.746000,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917000,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,55,149610,False,0.420,0.1660,1,-17.235001,1,0.0763,0.9240,0.000006,0.1010,0.267,77.488998,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,57,210826,False,0.438,0.3590,0,-9.734000,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332001,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,71,201933,False,0.266,0.0596,0,-18.514999,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740005,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,82,198853,False,0.618,0.4430,2,-9.681000,1,0.0526,0.4690,0.000000,0.0829,0.167,119.948997,4,acoustic


### QUERY Tasks

**1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99**

In [13]:
%%time
# For the sake of the table that I am using, I took out the artist_name and track_name due to columns
df = wr.athena.read_sql_query(sql="SELECT artists, popularity FROM {} WHERE popularity>=99".format(table_name_csv), database=database_name)

CPU times: user 584 ms, sys: 32.3 ms, total: 616 ms
Wall time: 2.74 s


In [14]:
df.head()

,artists,popularity
0,Sam Smith;Kim Petras,100
1,Bizarrap;Quevedo,99
2,Sam Smith;Kim Petras,100


**2. List artists with an average popularity of 92**

In [15]:
%%time
df = wr.athena.read_sql_query(sql="SELECT artists, AVG(popularity) AS avg_popularity FROM {}.{} GROUP BY artists HAVING AVG(popularity)=92".format(database_name,table_name_csv), database=database_name)

CPU times: user 465 ms, sys: 18 ms, total: 483 ms
Wall time: 3.29 s


In [16]:
df.head()

,artists,avg_popularity
0,Rema;Selena Gomez,92.0
1,Harry Styles,92.0


**3.List the Top 10 most energetic genres**

In [17]:
%%time
df = wr.athena.read_sql_query(sql="SELECT AVG(energy) as avg_energy, track_genre FROM {}.{} GROUP BY track_genre ORDER BY avg_energy DESC LIMIT 10".format(database_name,table_name_csv), database=database_name)

CPU times: user 600 ms, sys: 49.3 ms, total: 649 ms
Wall time: 2.96 s


In [18]:
df.head(10)

,avg_energy,track_genre
0,0.931470,death-metal
1,0.924201,grindcore
2,0.914220,metalcore
3,0.910971,happy
4,0.901246,hardstyle
5,0.876617,drum-and-bass
6,0.874897,black-metal
7,0.874003,heavy-metal
8,0.871237,party
9,0.868677,j-idol


**4. How many tracks is Bad Bunny On?**

In [19]:
%%time
df = wr.athena.read_sql_query(sql="SELECT COUNT(*) AS bb_count FROM {}.{} WHERE artists='Bad Bunny'".format(database_name,table_name_csv), database=database_name)
# For this particular scnenario, I used the artists feature column since I removed the track_name, I hope this is still acceptable. Thank you.

CPU times: user 555 ms, sys: 52.8 ms, total: 608 ms
Wall time: 3.48 s


In [20]:
df.head()

,bb_count
0,48


**5. Show the Top 10 genres in terms of popularity sorted by their most popular track**

In [21]:
%%time
df = wr.athena.read_sql_query(sql="SELECT MAX(popularity) as max_popularity, track_genre FROM {}.{} GROUP BY track_genre ORDER BY max_popularity DESC LIMIT 10".format(database_name,table_name_csv), database=database_name)

CPU times: user 615 ms, sys: 47.2 ms, total: 663 ms
Wall time: 2.98 s


In [22]:
df.head(10)

,max_popularity,track_genre
0,100,pop
1,100,dance
2,99,hip-hop
3,98,latino
4,98,edm
5,98,reggaeton
6,98,latin
7,98,reggae
8,96,piano
9,96,rock


### Rewritting this in Pandas

In [23]:
%store

Stored variables and their in-db values:
data_path                                         -> '/root/AAI-540/Module2/csv'
ingest_create_athena_db_mod2_passed               -> True
ingest_create_athena_table_csv_passed             -> True
s3_private_path_csv                               -> 's3://sagemaker-us-east-1-004608622582/module2_dat
setup_dependencies_mod2_passed                    -> True
setup_s3_bucket_passed                            -> True


In [24]:
%store -r data_path

In [25]:
# Reading in the CSV
import pandas as pd
df_pd = pd.read_csv(f"{data_path}/new_dataset.csv")
df_pd.head()

,track_id,artists,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


**1. List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99**

In [26]:
query_1 = df_pd[df_pd['popularity']>=99][['artists','popularity']]
print(query_1)         

                    artists  popularity
20001  Sam Smith;Kim Petras         100
51664      Bizarrap;Quevedo          99
81051  Sam Smith;Kim Petras         100


**2. List artists with an average popularity of 92**

In [27]:
avg = df_pd.groupby('artists')['popularity'].mean().reset_index()
query_2 = avg[avg['popularity']==92]
print(query_2)         

                 artists  popularity
11491       Harry Styles        92.0
22845  Rema;Selena Gomez        92.0


**3.List the Top 10 most energetic genres**

In [28]:
avg = df_pd.groupby('track_genre')['energy'].mean().reset_index()
query_3 = avg.sort_values(by='energy', ascending=False).head(10)
print(query_3)       

      track_genre    energy
22    death-metal  0.931470
42      grindcore  0.924201
72      metalcore  0.914485
46          happy  0.910971
49      hardstyle  0.901246
27  drum-and-bass  0.876635
6     black-metal  0.874897
50    heavy-metal  0.874003
78          party  0.871237
61         j-idol  0.868677


**4. How many tracks is Bad Bunny On?**

In [29]:
# For this particular scnenario, I used the artists feature column since I removed the track_name, I hope this is still acceptable. Thank you.
query_4 = sum(df_pd[df_pd['artists']=='Bad Bunny'].value_counts())
print(f"There are {query_4} Bad Bunny Tracks")  

There are 48 Bad Bunny Tracks


**5. Show the Top 10 genres in terms of popularity sorted by their most popular track**

In [30]:
avg = df_pd.groupby('track_genre')['popularity'].max().reset_index()
query_3 = avg.sort_values(by='popularity', ascending=False).head(10)
print(query_3)       

   track_genre  popularity
20       dance         100
80         pop         100
51     hip-hop          99
67       latin          98
30         edm          98
68      latino          98
89   reggaeton          98
88      reggae          98
90        rock          96
79       piano          96


Some of the values here are in slightly different order, but the contents match. Thank you!

### Release Resources

In [31]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>